In [ ]:
import boto3
from datetime import date
import time
import uuid
from credentials import *

## Connect to DynamoDB

Initially, we connect to our local database we started as "dynamodb" container:

In [ ]:
client = boto3.client('dynamodb', 
  endpoint_url='http://dynamo:8000', 
  region_name='eu-central-1',
  aws_access_key_id=aws_access_key_id,
  aws_secret_access_key=aws_secret_access_key
)
client.list_tables()

## Create Table

Let's setup the table with our attributes and indexes:

In [ ]:
def create_app_table():
    return client.create_table(
        TableName="twitter",
        KeySchema=[
            {"AttributeName": "PK", "KeyType": "HASH"},
            {"AttributeName": "SK", "KeyType": "RANGE"},
        ],
        AttributeDefinitions=[
            {"AttributeName": "PK", "AttributeType": "S"},
            {"AttributeName": "SK", "AttributeType": "S"},
        ],
        ProvisionedThroughput={"ReadCapacityUnits": 10, "WriteCapacityUnits": 10},
    )

In [ ]:
def delete_app_table():
    try:
        return client.delete_table(TableName="twitter")
    except:
        return False

In [ ]:
delete_app_table()
create_app_table()

## Helpers

To setup some other keys later on, we create a function to extract the id from a key:

In [ ]:
def parse_id_from_key(key):
    return key.split("#")[-1]

parse_id_from_key("CHAN#7")

## Users

We create and save our users:

In [ ]:
uuid_entry1 = str(uuid.uuid4())
user_martin = {
    'PK': {'S': f"USER#{uuid_entry1}"},
    'SK': {'S': f"USER#{uuid_entry1}"},
    'Name': {'S': 'Martin Marsal'},
}

uuid_entry2 = str(uuid.uuid4())
user_christian = {
    'PK': {'S': f"USER#{uuid_entry2}"},
    'SK': {'S': f"USER#{uuid_entry2}"},
    'Name': {'S': 'Christian Diegmann'},
}

uuid_entry3 = str(uuid.uuid4())
user_robin = {
    'PK': {'S': f"USER#{uuid_entry3}"},
    'SK': {'S': f"USER#{uuid_entry3}"},
    'Name': {'S': 'Robin Schüle'},
}

def save_user(user):
    return client.put_item(
        TableName="twitter",
        Item=user
    )

# AP6
def find_user(primaryKey, sortKey):
    item = client.get_item(
        TableName="twitter",
        Key={
          'PK': { 'S': primaryKey },
          'SK': { 'S': sortKey }
        }
    )
    return item['Item'] if 'Item' in item else False

In [ ]:
save_user(user_martin)
find_user(user_martin['PK']['S'], user_martin['SK']['S'])

In [ ]:
save_user(user_christian)
find_user(user_christian['PK']['S'], user_christian['SK']['S'])

In [ ]:
save_user(user_robin)
find_user(user_robin['PK']['S'], user_robin['SK']['S'])

In [ ]:
def add_to_followers(primaryKey, follower):
    response = client.put_item(
        TableName="twitter",
        Item={
            'PK': {'S': primaryKey},
            'SK': {'S': f"FOLLOWER#{parse_id_from_key(follower['PK']['S'])}"},
            'Name': {'S': follower['Name']['S']},
        },
        )
    if response['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.get_item(
            TableName='twitter',
            Key={
                'PK': {'S': primaryKey},
                'SK': {'S': f"FOLLOWER#{parse_id_from_key(follower['PK']['S'])}"},
            }
        )

In [ ]:
add_to_followers(user_martin['PK']['S'], user_christian)

In [ ]:
add_to_followers(user_christian['PK']['S'], user_robin)

In [ ]:
add_to_followers(user_robin['PK']['S'], user_martin)

## 1st access pattern: Post a tweet


In [ ]:
uuid_entry4 = str(uuid.uuid4())
timestamp = int(time.time())
tweet = {'id': {'S': f"{uuid_entry4}"}, 'text': {'S': 'Moin, moin.'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}}
uuid_entry5 = str(uuid.uuid4())
timestamp = int(time.time())
tweet_2 = {'id': {'S': f"{uuid_entry5}"}, 'text': {'S': 'MongoDB ist super!'}, 'likes': {'N': '0'}, 'CreatedAt': { 'N': str(timestamp)}}
newMartin = find_user(user_martin['PK']['S'], user_martin['SK']['S'])

def post_tweet(primaryKey, tweet):
    client.put_item(
        TableName='twitter',
        Item={
            'PK': {'S': primaryKey},
            'SK': {'S': f"TWEET#{tweet['id']['S']}"},
            'id': {'S': tweet['id']['S']},
            'text': {'S': tweet['text']['S']},
            'likes': {'N': tweet['likes']['N']},
            'CreatedAt': { 'N': tweet['CreatedAt']['N']}
        },
    )
    item = client.query(
        TableName="twitter",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :msg)',
        ExpressionAttributeValues={
            ':pk': { 'S': primaryKey },
            ':msg': { 'S': 'FOLLOWER#' }
        },
    )
    followers = item['Items']
    for follower in followers:
        response2 = client.put_item(
        TableName='twitter',
        Item={
            'PK': {'S': follower['SK']['S']},
            'SK': {'S': f"TIMELINE#{tweet['id']['S']}"},
            'tweetId': {'S': tweet['id']['S']},
            'text': {'S': tweet['text']['S']},
            'likes': {'N': tweet['likes']['N']},
            'CreatedAt': { 'N': tweet['CreatedAt']['N']}
        },
    )
    if response2['ResponseMetadata']['HTTPStatusCode'] == 200:
        return client.scan(
            TableName='twitter'
        )
        

In [ ]:
post_tweet(user_martin['PK']['S'], tweet)

In [ ]:
post_tweet(user_martin['PK']['S'], tweet_2)

## 3rd access pattern: Edit a tweet

In [ ]:
def edit_tweet(primaryKey, sortKey, tweet_id, new_text):
    
    user = find_user(primaryKey, sortKey)

    
    if user and 'Tweets' in user:
        tweets = user['Tweets']['L']

        
        tweet_index = None
        for i, tweet_item in enumerate(tweets):
            if tweet_item['M']['id']['S'] == tweet_id:
                tweet_index = i
                break

        
        if tweet_index is not None:
            tweets[tweet_index]['M']['text']['S'] = new_text

            response = client.update_item(
                TableName='twitter',
                Key={
                    'PK': {'S': primaryKey},
                    'SK': {'S': sortKey}
                },
                UpdateExpression='SET Tweets = :tweets',
                ExpressionAttributeValues={':tweets': {'L': tweets}},
                ReturnValues='UPDATED_NEW'
            )

            # Update timelines
            for follower in user['Followers']['L']:
                follower_user = find_user(user['PK']['S'], f"USER#{follower['S']}")
                if follower_user and 'Timeline' in follower_user:
                    timeline = follower_user['Timeline']['L']
                    if any(tweet['M']['id']['S'] == tweet_id for tweet in timeline):
                        
                        for tweet_item in timeline:
                            if tweet_item['M']['id']['S'] == tweet_id:
                                tweet_item['M']['text']['S'] = new_text
                                break

                        
                        client.update_item(
                            TableName='twitter',
                            Key={
                                'PK': {'S': user['PK']['S']},
                                'SK': {'S': f"USER#{follower['S']}"}
                            },
                            UpdateExpression='SET Timeline = :timeline',
                            ExpressionAttributeValues={':timeline': {'L': timeline}},
                            ReturnValues='UPDATED_NEW'
                        )

            return response
    else:
        return False

In [ ]:
old_tweet_id_to_edit = uuid_entry5  
new_tweet_text = "DynamoDB is doch besser"
edit_tweet(user_martin['PK']['S'], user_martin['SK']['S'], old_tweet_id_to_edit, new_tweet_text)

In [ ]:
find_user(user_martin['PK']['S'], user_martin['SK']['S'])

In [ ]:
find_user(user_christian['PK']['S'], user_christian['SK']['S'])

# 4th access pattern: Read a timeline

In [ ]:
def read_timeline(primaryKey, sortKey):
    
    user = find_user(primaryKey, sortKey)

    if user and 'Timeline' in user:
        return user['Timeline']['L']
    else:
        return []

In [ ]:
read_timeline(user_christian['PK']['S'], user_christian['SK']['S'])

## Channel

We create our channel and functions to save, find and count users and messages in the channel:

In [ ]:
channel_town_hall = {
    'PK': { 'S': "CHAN#7" },
    'SK': { 'S': "CHAN#7" },
    'Name': { 'S': 'Town Hall' },
    'Desc': { 'S': 'General News' },
    'UserCount': { 'N': "0" },
    'MessageCount': { 'N': "0" }
}

def save_channel(channel):
    return client.put_item(
        TableName="chat",
        Item=channel
    )

# AP2
def find_channel(key):
    item = client.get_item(
        TableName="chat",
        Key={
          'PK': { 'S': key },
          'SK': { 'S': key }
        }
    )
    return item['Item'] if 'Item' in item else False

# AP4
def message_count_for_channel(channel):
    return int(channel['MessageCount']['N'])

# AP7
def user_count_for_channel(channel):
    return int(channel['UserCount']['N'])

save_channel(channel_town_hall)
channel_town_hall = find_channel(channel_town_hall['PK']['S'])
channel_town_hall

In [ ]:
message_count_for_channel(channel_town_hall)

In [ ]:
user_count_for_channel(channel_town_hall)

## User Join

When a user joins a channel, we create a new item and increment the user counter. Further, we setup queries to retreive the users for a channel and channels of a user:

In [ ]:
def join_channel(user, channel):
    # create userj record
    client.put_item(
        TableName="chat",
        Item={
            'PK': channel['PK'],
            'SK': { 'S': "USERJ#" + parse_id_from_key(user['PK']['S']) },
            'Name': user['Name'],
            'JoinedAt': { 'S': str(date.today()) },
            'GSI1PK': user['PK'],
            'GSI1SK': channel['PK'],
            'ChanName': channel['Name']
        }
        )
    
    # increment users
    client.update_item(
        TableName="chat",
        Key = {
            'PK': channel['PK'],
            'SK': channel['PK'],
        },
        ExpressionAttributeValues = {
            ':one': { 'N': '1' }
        },
        UpdateExpression = 'ADD UserCount :one', 
        ReturnValues = 'UPDATED_NEW'
      )

# AP5
def users_in_channel(key):
    item = client.query(
        TableName="chat",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :userj)',
        ExpressionAttributeValues={
            ':pk': key,
            ':userj': { 'S': 'USERJ#' }
        }
    )
    return item['Items'] if 'Items' in item else []

# AP1
def channels_for_user(key):
    item = client.query(
        TableName="chat",
        IndexName="GSI1",
        KeyConditionExpression='GSI1PK = :pk AND begins_with(GSI1SK, :chan)',
        ExpressionAttributeValues={
            ':pk': key,
            ':chan': { 'S': 'CHAN#' }
        }
    )
    return item['Items'] if 'Items' in item else []

join_channel(user_alice, channel_town_hall)
join_channel(user_bob, channel_town_hall)
users_in_channel(channel_town_hall['PK'])

In [ ]:
channels_for_user(user_alice['PK'])

In [ ]:
find_channel(channel_town_hall['PK']['S'])

## Messages

When we send a message, we create a new item and increment the messages counter:

In [ ]:
def send_message(user, channel, text):
    timestamp = int(time.time())
    message_id = uuid.uuid4()
    
    # create message
    client.put_item(
        TableName="chat",
        Item={
            'PK': channel['PK'],
            'SK': { 'S': f"MSG#{timestamp}#{message_id}" },
            'Msg': { 'S': text },
            'CreatedAt': { 'N': str(timestamp) },
            'UserName': user['Name'],
            'UserId': user['PK']
        }
        )
    
    # increment users
    client.update_item(
        TableName="chat",
        Key = {
            'PK': channel['PK'],
            'SK': channel['PK'],
        },
        ExpressionAttributeValues = {
            ':one': { 'N': '1' }
        },
        UpdateExpression = 'ADD MessageCount :one', 
        ReturnValues = 'UPDATED_NEW'
      )

# AP3
def messages_in_channel(key, limit = 50):
    item = client.query(
        TableName="chat",
        KeyConditionExpression='PK = :pk AND begins_with(SK, :msg)',
        ExpressionAttributeValues={
            ':pk': key,
            ':msg': { 'S': 'MSG#' }
        },
        Limit = limit
    )
    return item['Items'] if 'Items' in item else []
    

send_message(user_bob, channel_town_hall, "Hey there!")
send_message(user_alice, channel_town_hall, "Hello Bob!")
send_message(user_bob, channel_town_hall, "How are you?")
messages_in_channel(channel_town_hall['PK'])

In [ ]:
find_channel(channel_town_hall['PK']['S'])